In [ ]:
# run whole script
%run 03_safe_execution.ipynb

In [24]:
import yaml
import json
import os
import sys
from pathlib import Path
from typing import Tuple, Dict, Any, TypedDict, Optional, List
from dotenv import load_dotenv

# LangChain and LangGraph
from langgraph.graph import StateGraph, END 
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

True

In [25]:
# Retry and attempt limits
MAX_RETRIES = 2  # Max retries per strategy
MAX_TOTAL_ATTEMPTS = 6  # Total attempts across all strategies

## Database Connection

In [26]:
import sys
from pathlib import Path
sys.path.append(str(Path("D:/code/text-to-sql-agent")))

from src.db.db_connection import get_db_connection

conn = get_db_connection()
cursor = conn.cursor()

print("✅ Database connected")

✅ Database connected


## Load Schema

In [ ]:
SCHEMA_PATH = Path("../src/schema/schema_summary.yaml")

with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    FULL_SCHEMA: Dict[str, Any] = yaml.safe_load(f)

print(f"✅ Loaded schema with {len(FULL_SCHEMA['tables'])} tables")

✅ Loaded schema with 6 tables


## LLM Configuration

In [28]:
def load_llm():
    """Initialize OpenAI LLM"""
    return ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )

def call_llm(prompt: str) -> str:
    """Call LLM with prompt and return response"""
    llm = load_llm()
    response = llm.invoke(prompt)
    return response.content.strip()


In [29]:
class SQLAgentState(TypedDict):
    """State schema for the SQL agent"""
    # Core fields
    question: str
    sql: Optional[str]
    valid: bool
    reason: Optional[str]
    
    # Execution tracking
    executed: bool
    results: Optional[list]
    nl_response: Optional[str]
    
    # Strategy tracking
    retries: int
    total_attempts: int
    failure_type: Optional[str]
    attempted_strategies: List[str]
    current_strategy: str
    
    # Planning fields
    planned_tables: Optional[List[str]]
    filtered_schema: Optional[Dict[str, Any]]

In [30]:
def ensure_schema_dict(schema: Any) -> Dict[str, Any]:
    """Ensure schema is a dict, fallback to FULL_SCHEMA if not"""
    if isinstance(schema, dict):
        return schema
    print(f"⚠️ Warning: Invalid schema type {type(schema)}, using FULL_SCHEMA")
    return FULL_SCHEMA

## Tool: Schema Filter

In [31]:
def schema_filter_tool(table_names: List[str]) -> Dict[str, Any]:
    """
    Extract only specified tables and their related info from full schema.
    This is the 'tool' the agent uses to get relevant schema.
    """
    filtered_schema = {
        'tables': {},
        'hints': FULL_SCHEMA.get('hints', []),
        'common_joins': []
    }
    
    # Extract requested tables
    for table_name in table_names:
        if table_name in FULL_SCHEMA['tables']:
            filtered_schema['tables'][table_name] = FULL_SCHEMA['tables'][table_name]
    
    # Extract relevant joins (any join involving requested tables)
    for join_info in FULL_SCHEMA.get('common_joins', []):
        join_tables = set(join_info.get('tables', []))
        if join_tables.intersection(set(table_names)):
            filtered_schema['common_joins'].append(join_info)
    
    return filtered_schema

## Prompts

In [32]:
def build_planning_prompt(question: str) -> str:
    """Prompt for planning node to decide which tables are needed"""
    available_tables = list(FULL_SCHEMA['tables'].keys())
    table_descriptions = {
        name: FULL_SCHEMA['tables'][name].get('description', 'No description')
        for name in available_tables
    }
    
    return f"""
You are a database query planner. Analyze the user's question and decide which tables are needed.

Available tables:
{json.dumps(table_descriptions, indent=2)}

User question:
{question}

Your task:
1. Identify which tables are needed to answer this question
2. Consider foreign key relationships (you may need junction tables)
3. Output ONLY a JSON array of table names

Rules:
- Include ALL tables needed for joins
- If asking about products AND orders, include order_products_* tables
- Don't include unnecessary tables
- Output ONLY valid JSON, no explanation

Example outputs:
["products"]
["orders", "order_products_prior", "products"]
["products", "aisles", "departments"]

JSON array of table names:
""".strip()


def build_optimized_prompt(question: str, schema: Dict[str, Any]) -> str:
    """Optimized SQL generation prompt"""
    return f"""
You are an expert PostgreSQL SQL generator.

CRITICAL RULES:
- Output ONLY one SQL SELECT query
- Do NOT include markdown, backticks, or explanations
- Use ONLY tables and columns from the schema
- Follow join templates strictly
- Never invent joins or columns
- Prefer correctness over brevity

Database schema with semantics:
{schema}

User question:
{question}

SQL:
""".strip()


def build_optimized_correction_prompt(
    question: str,
    schema: Dict[str, Any],
    previous_sql: str,
    error_reason: str
) -> str:
    """Optimized correction prompt with available columns list"""
    
    # Safety check
    if not isinstance(schema, dict):
        print(f"⚠️ Warning: schema is {type(schema)}, using FULL_SCHEMA")
        schema = FULL_SCHEMA
    
    # Extract actual available columns SAFELY
    available_columns = []
    tables = schema.get('tables', {})
    
    if isinstance(tables, dict):
        for table_name, table_info in tables.items():
            if isinstance(table_info, dict):
                columns = table_info.get('columns', [])
                if isinstance(columns, list):
                    for col in columns:
                        if isinstance(col, dict) and 'name' in col:
                            available_columns.append(f"{table_name}.{col['name']}")
    
    # Fallback if no columns found
    if not available_columns:
        available_columns = ["Unable to extract column list"]
    
    return f"""
You are an expert PostgreSQL SQL generator. The previous SQL query failed.

Failure reason:
{error_reason}

⚠️ CRITICAL: If the error mentions columns that DON'T EXIST, you must NOT use them.

AVAILABLE COLUMNS ONLY:
{', '.join(available_columns[:50])}

ABSOLUTE REQUIREMENTS:
1. Use ONLY columns listed above - do NOT invent columns
2. If the question asks for data we don't have (profit, margin, price), return a message:
   SELECT 'Data not available in database' as message
3. Output ONLY valid SELECT query with existing columns
4. No markdown, no backticks, no explanations

Original question:
{question}

Failed SQL:
{previous_sql}

Corrected SELECT query (using ONLY existing columns):
""".strip()


def build_validation_and_response_prompt(
    question: str,
    sql: str,
    results: list
) -> str:
    """
    Creates a prompt for LLM to:
    1. Validate if results answer the question
    2. Generate a natural language response
    """
    sample_results = results[:10]
    total_rows = len(results)
    
    return f"""
You are a SQL result validator and response generator.

Your tasks:
1. Validate if the SQL query results actually answer the user's question
2. Generate a natural, conversational answer for the user

CRITICAL RULES:
- Output ONLY valid JSON with three fields: "valid" (boolean), "reason" (string), "natural_language_response" (string)
- For validation: Check if query targeted RIGHT tables/columns and results are plausible
- For response: Be conversational, clear, and directly answer the question
- Include relevant numbers/data from results
- Don't mention SQL or technical details in the natural language response
- If results are empty or wrong, explain what went wrong in a user-friendly way

User Question:
{question}

SQL Query Executed:
{sql}

Results (showing {len(sample_results)} of {total_rows} total rows):
{sample_results}

Validation Checklist:
1. Does the SQL query the correct tables/columns for this question?
2. Do the returned values semantically match what was asked?
3. Are the results plausible? (no negative counts, reasonable magnitudes)

Response Guidelines:
- Be direct and conversational
- Use numbers from the results
- Format large numbers readably (e.g., "49,688" not "49688")
- If multiple rows, summarize or show top results
- Don't say "according to the query" - just answer

Output JSON format:
{{
    "valid": true/false,
    "reason": "brief explanation of validation decision",
    "natural_language_response": "conversational answer to user's question"
}}

JSON:
""".strip()


def build_simplified_prompt(
    question: str,
    schema: Dict[str, Any],
    previous_sql: str,
    error_reason: str
) -> str:
    """Generate a simpler query approach"""
    return f"""
You are an expert PostgreSQL SQL generator.

The previous complex query failed. Generate a SIMPLER query.

CRITICAL RULES:
- Use the SIMPLEST approach possible
- Avoid complex joins if possible
- Use single table if feasible
- Output ONLY SQL, no markdown

Database schema:
{schema}

User question:
{question}

Previous failed SQL (too complex):
{previous_sql}

Failure reason:
{error_reason}

Generate SIMPLER SQL:
""".strip()


def build_alternative_prompt(
    question: str,
    schema: Dict[str, Any],
    previous_sql: str,
    error_reason: str,
    attempted_strategies: List[str]
) -> str:
    """Generate a completely different approach"""
    return f"""
You are an expert PostgreSQL SQL generator.

Previous attempts failed. Think differently about this problem.

CRITICAL RULES:
- Approach this from a DIFFERENT angle
- Consider alternative tables or join patterns
- Use different aggregation methods if applicable
- Output ONLY SQL, no markdown

Database schema:
{schema}

User question:
{question}

Previous attempts that failed:
{attempted_strategies}

Previous SQL:
{previous_sql}

Failure reason:
{error_reason}

Generate ALTERNATIVE SQL approach:
""".strip()


In [8]:
# # -------------------------------------------------------
# # Generate SQL (optimized)
# # -------------------------------------------------------

def generate_sql_optimized(question: str) -> str:
    prompt = build_optimized_prompt(question, FULL_SCHEMA)
    raw_sql = call_llm(prompt)
    return raw_sql


In [33]:
def planning_node(state: SQLAgentState) -> SQLAgentState:
    """
    Analyzes question and decides which tables are needed.
    This is what makes it AGENTIC - the agent plans before acting.
    """
    print("🧠 Planning: Analyzing question...")
    
    prompt = build_planning_prompt(state["question"])
    response = call_llm(prompt)
    
    try:
        # Parse the JSON response
        planned_tables = json.loads(response)
        
        if not isinstance(planned_tables, list):
            print("⚠️ Planning failed: Invalid response format")
            planned_tables = list(FULL_SCHEMA['tables'].keys())
        
        print(f"📋 Plan: Need tables {planned_tables}")
        
        # Use schema filter tool to get relevant schema
        filtered_schema = schema_filter_tool(planned_tables)
        print(f"✂️ Filtered schema: {len(filtered_schema['tables'])} tables, {len(filtered_schema['common_joins'])} joins")
        
        return {
            **state, 
            "planned_tables": planned_tables,
            "filtered_schema": filtered_schema
        }
    
    except json.JSONDecodeError as e:
        print(f"⚠️ Planning failed to parse JSON: {e}")
        print(f"Raw response: {response[:200]}")
        return {
            **state,
            "planned_tables": list(FULL_SCHEMA['tables'].keys()),
            "filtered_schema": FULL_SCHEMA
        }


def generate_sql_node(state: SQLAgentState) -> SQLAgentState:
    """Generates SQL using filtered schema from planning node"""
    print("🔄 Generating SQL...")
    
    schema_to_use = ensure_schema_dict(state.get("filtered_schema", FULL_SCHEMA))
    
    prompt = build_optimized_prompt(state["question"], schema_to_use)
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    
    print(f"Generated: {sql[:100]}...")
    
    return {
        **state, 
        "sql": sql,
        "total_attempts": state.get("total_attempts", 0) + 1
    }


def validate_sql_node(state: SQLAgentState) -> SQLAgentState:
    """Validate SQL syntax and safety before execution"""
    print("🔍 Validating syntax...")
    is_valid, reason = validate_sql(state["sql"])
    print(f"Syntax valid: {is_valid}")
    if not is_valid:
        print(f"Reason: {reason}")
    return {
        **state, 
        "valid": is_valid, 
        "reason": None if is_valid else reason, 
        "executed": False
    }


def execute_sql_node(state: SQLAgentState) -> SQLAgentState:
    """Execute SQL with proper transaction management"""
    print("⚡ Executing SQL...")
    try:
        cursor.execute(state["sql"])
        results = cursor.fetchall()
        conn.commit()
        print(f"✅ Executed! Got {len(results)} rows")
        return {
            **state, 
            "executed": True, 
            "results": results, 
            "reason": None
        }
    except Exception as e:
        conn.rollback()
        print(f"❌ Execution failed: {str(e)[:100]}")
        print("🔄 Transaction rolled back")
        return {
            **state, 
            "executed": False, 
            "results": None, 
            "reason": f"Execution error: {str(e)}"
        }


def validate_and_respond_node(state: SQLAgentState) -> SQLAgentState:
    """Validates if SQL results answer the question AND generates natural language response"""
    print("🔍 Validating answer + generating response...")
    
    if not state["executed"] or not state["results"]:
        print("⚠️ Cannot validate - no results to check")
        return {
            **state, 
            "valid": False, 
            "reason": "No results to validate",
            "nl_response": "I couldn't execute the query to get an answer."
        }
    
    prompt = build_validation_and_response_prompt(
        question=state["question"],
        sql=state["sql"],
        results=state["results"]
    )
    
    response = call_llm(prompt)
    
    try:
        output = json.loads(response)
        
        is_valid = output.get("valid", False)
        reason = output.get("reason", "Unknown validation failure")
        nl_response = output.get("natural_language_response", "Unable to generate response.")
        
        if is_valid:
            print("✅ Answer validated! Generated NL response.")
            print(f"📝 Response: {nl_response[:100]}...")
        else:
            print(f"❌ Answer validation failed: {reason}")
        
        return {
            **state, 
            "valid": is_valid, 
            "reason": None if is_valid else reason,
            "nl_response": nl_response
        }
    
    except json.JSONDecodeError as e:
        print(f"⚠️ Failed to parse validation response: {e}")
        return {
            **state, 
            "valid": False, 
            "reason": "Validation parsing error",
            "nl_response": "Error processing the query results."
        }


def correct_sql_node(state: SQLAgentState) -> SQLAgentState:
    """Attempt to correct SQL based on error"""
    print(f"🔧 Correcting SQL (attempt {state['total_attempts'] + 1})...")
    
    schema_to_use = ensure_schema_dict(state.get("filtered_schema", FULL_SCHEMA))
    
    prompt = build_optimized_correction_prompt(
        question=state["question"],
        schema=schema_to_use,
        previous_sql=state["sql"],
        error_reason=state["reason"]
    )
    corrected_sql = call_llm(prompt)
    sql = clean_sql(corrected_sql)
    print(f"Corrected: {sql[:100]}...")
    
    attempted = state.get("attempted_strategies", [])
    if "correct" not in attempted:
        attempted.append("correct")
    
    return {
        **state, 
        "sql": sql, 
        "retries": state["retries"] + 1,
        "total_attempts": state.get("total_attempts", 0) + 1,
        "attempted_strategies": attempted
    }


def analyze_failure_node(state: SQLAgentState) -> SQLAgentState:
    """AGENTIC: Agent analyzes WHY it failed and classifies failure type"""
    print("🧠 Analyzing failure...")
    
    reason = state.get("reason", "")
    
    if "syntax" in reason.lower() or "invalid" in reason.lower():
        failure_type = "syntax_error"
        print("📊 Failure type: Syntax error")
    elif "no results" in reason.lower() or "empty" in reason.lower():
        failure_type = "no_results"
        print("📊 Failure type: No results returned")
    elif "execution" in reason.lower() or "does not exist" in reason.lower():
        failure_type = "execution_error"
        print("📊 Failure type: Execution error")
    elif not state.get("valid", False) and state.get("executed", False):
        failure_type = "semantic_error"
        print("📊 Failure type: Answer doesn't match question")
    else:
        failure_type = "unknown"
        print("📊 Failure type: Unknown")
    
    return {**state, "failure_type": failure_type}


def generate_simplified_sql_node(state: SQLAgentState) -> SQLAgentState:
    """STRATEGY: Try a simpler query approach"""
    print("🔄 Strategy: Generating SIMPLIFIED SQL...")
    
    schema_to_use = ensure_schema_dict(state.get("filtered_schema", FULL_SCHEMA))
    
    prompt = build_simplified_prompt(
        question=state["question"],
        schema=schema_to_use,
        previous_sql=state["sql"],
        error_reason=state["reason"]
    )
    
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    print(f"Simplified: {sql[:100]}...")
    
    attempted = state.get("attempted_strategies", [])
    attempted.append("simplified")
    
    return {
        **state, 
        "sql": sql, 
        "current_strategy": "simplified",
        "attempted_strategies": attempted,
        "retries": 0,
        "total_attempts": state.get("total_attempts", 0) + 1
    }


def generate_alternative_approach_node(state: SQLAgentState) -> SQLAgentState:
    """STRATEGY: Try a completely different approach"""
    print("🔄 Strategy: Trying ALTERNATIVE approach...")
    
    schema_to_use = ensure_schema_dict(state.get("filtered_schema", FULL_SCHEMA))
    
    prompt = build_alternative_prompt(
        question=state["question"],
        schema=schema_to_use,
        previous_sql=state["sql"],
        error_reason=state["reason"],
        attempted_strategies=state.get("attempted_strategies", [])
    )
    
    raw_sql = call_llm(prompt)
    sql = clean_sql(raw_sql)
    print(f"Alternative: {sql[:100]}...")
    
    attempted = state.get("attempted_strategies", [])
    attempted.append("alternative")
    
    return {
        **state, 
        "sql": sql, 
        "current_strategy": "alternative",
        "attempted_strategies": attempted,
        "retries": 0,
        "total_attempts": state.get("total_attempts", 0) + 1
    }


def ask_clarification_node(state: SQLAgentState) -> SQLAgentState:
    """FINAL FALLBACK: Agent admits it needs help and asks user"""
    print("💬 Strategy: Asking user for clarification...")
    
    clarification = f"""I tried multiple approaches but couldn't answer your question: "{state['question']}"

Attempts made: {state.get('total_attempts', 0)}
Strategies tried: {', '.join(state.get('attempted_strategies', ['direct']))}

Last error: {state['reason']}

Could you please:
- Rephrase your question, or
- Provide more specific details, or
- Break it into smaller questions?"""
    
    return {
        **state,
        "nl_response": clarification,
        "valid": False
    }

In [34]:
def route_after_syntax_check(state: SQLAgentState):
    """Route after pre-execution validation"""
    if state["valid"]:
        return "execute_sql"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"


def route_after_execution(state: SQLAgentState):
    """Route after execution - goes to combined validation+response node"""
    if state["executed"]:
        return "validate_and_respond"
    if state["retries"] >= MAX_RETRIES:
        return END
    return "correct_sql"


def route_after_validation(state: SQLAgentState):
    """Route after answer validation"""
    if state["valid"]:
        return END  # Success!
    return "analyze_failure"


def route_after_failure_analysis(state: SQLAgentState):
    """AGENTIC ROUTING: Agent chooses strategy based on failure analysis"""
    attempted = state.get("attempted_strategies", [])
    failure_type = state.get("failure_type", "unknown")
    total_attempts = state.get("total_attempts", 0)
    
    print(f"🤔 Agent deciding next move...")
    print(f"   Total attempts: {total_attempts}/{MAX_TOTAL_ATTEMPTS}")
    print(f"   Strategies tried: {attempted}")
    print(f"   Failure type: {failure_type}")
    
    # PRIMARY ESCAPE HATCH - prevents infinite loops
    if total_attempts >= MAX_TOTAL_ATTEMPTS:
        print("   ⛔ Maximum attempts exhausted - asking user")
        return "ask_clarification"
    
    # Strategy escalation based on attempts
    if "simplified" not in attempted and total_attempts >= 2:
        print("   Decision: Try simplified SQL approach")
        return "generate_simplified"
    
    if "alternative" not in attempted and total_attempts >= 4:
        print("   Decision: Try completely different approach")
        return "generate_alternative"
    
    # Default: try correction if we have attempts left
    if total_attempts < MAX_TOTAL_ATTEMPTS - 1:
        print("   Decision: Try correcting SQL")
        return "correct_sql"
    
    # Final fallback
    print("   Decision: Give up, ask user for help")
    return "ask_clarification"

In [35]:
# Initialize graph
graph = StateGraph(SQLAgentState)

# Add all nodes
graph.add_node("planning", planning_node)
graph.add_node("generate_sql", generate_sql_node)
graph.add_node("validate_sql", validate_sql_node)
graph.add_node("execute_sql", execute_sql_node)
graph.add_node("validate_and_respond", validate_and_respond_node)
graph.add_node("correct_sql", correct_sql_node)
graph.add_node("analyze_failure", analyze_failure_node)
graph.add_node("generate_simplified", generate_simplified_sql_node)
graph.add_node("generate_alternative", generate_alternative_approach_node)
graph.add_node("ask_clarification", ask_clarification_node)

# Set entry point
graph.set_entry_point("planning")

# Define edges
graph.add_edge("planning", "generate_sql")
graph.add_edge("generate_sql", "validate_sql")

# Conditional edges
graph.add_conditional_edges(
    "validate_sql",
    route_after_syntax_check,
    {
        "execute_sql": "execute_sql",
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_conditional_edges(
    "execute_sql",
    route_after_execution,
    {
        "validate_and_respond": "validate_and_respond",
        "correct_sql": "correct_sql",
        END: END
    }
)

graph.add_conditional_edges(
    "validate_and_respond",
    route_after_validation,
    {
        "analyze_failure": "analyze_failure",
        END: END
    }
)

graph.add_conditional_edges(
    "analyze_failure",
    route_after_failure_analysis,
    {
        "correct_sql": "correct_sql",
        "generate_simplified": "generate_simplified",
        "generate_alternative": "generate_alternative",
        "ask_clarification": "ask_clarification"
    }
)

# Strategy nodes back to validation
graph.add_edge("generate_simplified", "validate_sql")
graph.add_edge("generate_alternative", "validate_sql")
graph.add_edge("correct_sql", "validate_sql")
graph.add_edge("ask_clarification", END)

# Compile the graph
sql_agent = graph.compile()

print("✅ Graph compiled successfully")


✅ Graph compiled successfully


In [36]:
# RUN AGENT - TEST 1: IMPOSSIBLE QUESTION

print("\n" + "="*70)
print("TEST 1: IMPOSSIBLE QUESTION (Stress Test)")
print("="*70)

initial_state: SQLAgentState = {
    "question": "What's the average profit margin per product category last month?",
    "sql": None,
    "valid": False,
    "reason": None,
    "retries": 0,
    "executed": False,
    "results": None,
    "nl_response": None,
    "failure_type": None,
    "attempted_strategies": [],
    "current_strategy": "direct",
    "planned_tables": None,
    "filtered_schema": None,
    "total_attempts": 0
}

# Run the agent
final_state = sql_agent.invoke(
    initial_state,
    config={"recursion_limit": 50}  # Increased from default 25
)

# Display results
print("\n" + "="*70)
print("🤖 AGENTIC SQL AGENT RESULTS - TEST 1")
print("="*70)

if final_state["valid"] and final_state["executed"]:
    print("\n✅ SUCCESS")
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n🧠 Planned Tables: {final_state['planned_tables']}")
    print(f"\n💬 Answer: {final_state['nl_response']}")
    print(f"\n🔧 SQL: {final_state['sql']}")
    print(f"\n📊 Raw Results (sample): {final_state['results'][:3]}")
    print(f"\n📈 Total Attempts: {final_state['total_attempts']}")
else:
    print("\n❌ FAILED (Expected for impossible question)")
    print(f"\n🙋 Question: {final_state['question']}")
    print(f"\n🧠 Planned Tables: {final_state.get('planned_tables', 'Not planned')}")
    print(f"\n⚠️ Error: {final_state['reason']}")
    print(f"\n💬 Response: {final_state['nl_response']}")
    print(f"\n📈 Total Attempts: {final_state['total_attempts']}/{MAX_TOTAL_ATTEMPTS}")
    print(f"\n🔄 Strategies Tried: {', '.join(final_state['attempted_strategies'])}")

print("="*70)


TEST 1: IMPOSSIBLE QUESTION (Stress Test)
🧠 Planning: Analyzing question...
📋 Plan: Need tables ['orders', 'order_products_prior', 'products', 'departments']
✂️ Filtered schema: 4 tables, 5 joins
🔄 Generating SQL...
Generated: select p.product_id, avg(op.reordered) as average_profit_margin from orders o inner join order_produ...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
❌ Execution failed: unit "hour" not supported for type date

🔄 Transaction rolled back
🔧 Correcting SQL (attempt 2)...
Corrected: select 'data not available in database' as message...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
✅ Executed! Got 1 rows
🔍 Validating answer + generating response...
❌ Answer validation failed: The query indicates that data is not available in the database, which means it did not target the correct tables or columns to retrieve the average profit margin per product category.
🧠 Analyzing failure...
📊 Failure type: Answer doesn't match question
🤔 Agent decid

In [37]:
# RUN AGENT - TEST 2: NORMAL QUESTION

print("\n" + "="*70)
print("TEST 2: NORMAL QUESTION")
print("="*70)

normal_state: SQLAgentState = {
    "question": "Show me the top 5 most ordered products",
    "sql": None,
    "valid": False,
    "reason": None,
    "retries": 0,
    "executed": False,
    "results": None,
    "nl_response": None,
    "failure_type": None,
    "attempted_strategies": [],
    "current_strategy": "direct",
    "planned_tables": None,
    "filtered_schema": None,
    "total_attempts": 0
}

normal_result = sql_agent.invoke(normal_state)

print("\n" + "="*70)
print("🤖 AGENTIC SQL AGENT RESULTS - TEST 2")
print("="*70)

if normal_result["valid"] and normal_result["executed"]:
    print("\n✅ SUCCESS")
    print(f"\n🙋 Question: {normal_result['question']}")
    print(f"\n🧠 Planned Tables: {normal_result['planned_tables']}")
    print(f"\n💬 Answer: {normal_result['nl_response']}")
    print(f"\n🔧 SQL: {normal_result['sql']}")
    print(f"\n📊 Raw Results (sample): {normal_result['results'][:3]}") 
    print(f"\n📈 Total Attempts: {normal_result['total_attempts']}") 
else: 
    print("\n❌ FAILED") 
    print(f"\n🙋 Question: {normal_result['question']}") 
    print(f"\n🧠 Planned Tables: {normal_result.get('planned_tables', 'Not planned')}") 
    print(f"\n⚠️ Error: {normal_result['reason']}") 
    print(f"\n💬 Response: {normal_result['nl_response']}") 
    print(f"\n📈 Total Attempts: {normal_result['total_attempts']}/{MAX_TOTAL_ATTEMPTS}") 
    print(f"\n🔄 Strategies Tried: {', '.join(normal_result['attempted_strategies'])}")

print("="*70)


TEST 2: NORMAL QUESTION
🧠 Planning: Analyzing question...
📋 Plan: Need tables ['order_products_prior', 'products']
✂️ Filtered schema: 2 tables, 5 joins
🔄 Generating SQL...
Generated: select op.product_id, count(op.order_id) as order_count from order_products_prior op group by op.pro...
🔍 Validating syntax...
Syntax valid: True
⚡ Executing SQL...
✅ Executed! Got 5 rows
🔍 Validating answer + generating response...
✅ Answer validated! Generated NL response.
📝 Response: Here are the top 5 most ordered products: Product ID 24852 has been ordered 472,565 times, followed ...

🤖 AGENTIC SQL AGENT RESULTS - TEST 2

✅ SUCCESS

🙋 Question: Show me the top 5 most ordered products

🧠 Planned Tables: ['order_products_prior', 'products']

💬 Answer: Here are the top 5 most ordered products: Product ID 24852 has been ordered 472,565 times, followed by Product ID 13176 with 379,450 orders. Next is Product ID 21137 with 264,683 orders, then Product ID 21903 at 241,921 orders, and finally, Product ID 47